In [ ]:
from bs4 import BeautifulSoup
import bs4 as bs
import urllib.request
import pandas as pd
import numpy as np
import re
import requests

In [ ]:
first_third=pd.read_csv('first_third_of_data.csv')
second_third=pd.read_csv('second_third_of_data.csv')
last_third=pd.read_csv('last_third_of_data.csv')

In [ ]:
data=pd.concat([first_third, second_third,last_third], ignore_index=True)
data.drop('Unnamed: 0', axis=1, inplace=True)

Start with webscraping all of the New York Time recipe pages that are referenced in the Kaggle Dataset. The data that we are interested in is the recipe ratings and the number of ingredients that each recipe requires.

In [ ]:
def create_urls(end, urls):
    base_url='https://cooking.nytimes.com/recipes/'+end
    urls.append(base_url)

In [ ]:
end_url=[]

group_df=data.groupby(['recipe_id','recipe_name']).sum()
group_df=pd.DataFrame(group_df)
group_df=group_df.reset_index()


id_dict={}

for row in range(len(group_df)):
    recipe_id=str(group_df.loc[row,'recipe_id'])
    id_dict[recipe_id]=[]
    recipe_name=group_df.loc[row,'recipe_name']
    recipe_name=recipe_name.replace(" ","-")
    end_url.append(recipe_id+'-'+recipe_name)
    
urls=[]    
for end in end_url:
    create_urls(end, urls)

In [ ]:
def scrape_recipe_page(main_url):
    soup = BeautifulSoup(requests.get(main_url).text, "html.parser")
    
    #Scrape recipe rating
    try:
        rating_text=soup.find("script", type="application/ld+json").text
        begin_pos=rating_text.find("ratingValue")
        rating_text=rating_text[begin_pos:]
        end_pos=rating_text.find(":")
        rating=rating_text[end_pos+1:end_pos+2]
    except:
        rating=-1

    #Scrape the number of ingredients
    num_ingredients=len(soup.find_all("span", class_="ingredient-name"))
    
    #Add back to dict to later join with dataframe
    pos_num=re.search(r"\d", main_url)
    if pos_num:
        end_num=main_url[pos_num.start():].find('-')
        if len(id_dict[main_url[pos_num.start():pos_num.start()+end_num]])<2:
            id_dict[main_url[pos_num.start():pos_num.start()+end_num]].extend([num_ingredients,rating])
        else:
            pass  

In [ ]:
for url in urls:
    scrape_recipe_page(url)

In [ ]:
ingredient_rating_df=pd.DataFrame.from_dict(id_dict, orient='index',
                       columns=['num_ingredients', 'rating'])

ingredient_rating_df.index.name='recipe_id'      
ingredient_rating_df

Write webscraped data to a csv file so we do not have to re-run webscraper everytime we open notebook.

In [ ]:
ingredient_rating_df.to_csv('ingredient_rating_df.csv')